<a href="https://colab.research.google.com/github/thamyrissacamoto/MarvelAPI/blob/Thamyris%2FconsumindoAPI/client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dotenv

In [2]:
import hashlib
import time
import requests
from google.colab import userdata
import pandas as pd
from tqdm import tqdm
import os
from dotenv import load_dotenv

In [4]:
BASE_URL = "https://gateway.marvel.com/v1/public/"

# load_dotenv()
# PUBLIC_KEY = os.getenv("public_key")
# PRIVATE_KEY = os.getenv("private_key")

PUBLIC_KEY = userdata.get('public_key')
PRIVATE_KEY = userdata.get('private_key')

def get_auth():
    # Gera o hash md5 de autenticação

    ts = str(time.time())
    to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
    hash_result = hashlib.md5(to_hash.encode()).hexdigest()
    return {
        "ts": ts,
        "apikey": PUBLIC_KEY,
        "hash": hash_result
    }


In [11]:
etag_cache = {}

def marvel_request(endpoint, params_extra={}):
    # Faz uma requisição GET com cache ETag

    url = BASE_URL + endpoint
    auth_params = get_auth()
    params = auth_params.copy()
    params.update(params_extra)

    cache_key = f"{endpoint}_{params_extra}"
    headers = {}
    if cache_key in etag_cache:
        headers["If-None-Match"] = etag_cache[cache_key]["etag"]

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 304: # Evita refazer uma requisição
        return etag_cache[cache_key]["data"]
    elif response.status_code == 200:
        etag = response.headers.get("etag")
        data = response.json()
        etag_cache[cache_key] = {"etag": etag, "data": data}
        return data
    else:
        print(f"Erro {response.status_code}: {response.text}")
        return None

In [12]:
def get_all_data(endpoint):
    # Extrai dados do endpoint principal

    first_response = marvel_request(endpoint, {"limit": 100, "offset": 0})
    if not first_response:
        print("Erro ao obter dados iniciais.")
        return None

    total = first_response["data"]["total"]
    all_results = first_response["data"]["results"]

    for offset in tqdm(range(100, total, 100), desc=f"Baixando dados"):
        response = marvel_request(endpoint, {"limit": 100, "offset": offset})
        if response:
            all_results.extend(response["data"]["results"])
        time.sleep(0.3)

    # Normaliza e salva em CSV
    df = pd.json_normalize(all_results)
    df.to_csv(f"{endpoint}.csv", index=False)
    return df

In [13]:
def get_relationships(endpoint, ids, relationships):
    # Extrai relacionamentos por IDs

    results = []

    for item_id in tqdm(ids, desc="Buscando relacionamentos"):
        item_data = {"id": item_id}

        for rel in relationships:
            rel_url = f"{endpoint}/{item_id}/{rel}"
            rel_response = marvel_request(rel_url)
            if rel_response:
                rel_ids = [r["id"] for r in rel_response["data"]["results"]]
                item_data[f"{rel}_ids"] = rel_ids
            else:
                item_data[f"{rel}_ids"] = []
            time.sleep(0.2)

        results.append(item_data)

    df = pd.DataFrame(results)
    df.to_csv(f"{endpoint}_ids_relationships.csv", index=False)
    return df


In [15]:
# Endpoints
endpoints = ["characters", "comics", "events", "series", "stories"]

for ep in endpoints:
    print(f"\nExtraindo dados de: {ep}")
    get_all_data(ep)

# IDs para relacionamento
ids = [1009610, 1009726, 1009718, 1009368, 1009220, 1009165, 1009664, 1009351, 1009299, 1009262]
df_rels = get_relationships("characters", ids, relationships=["comics", "series", "events", "stories"])
print("\nRelacionamentos extraídos com sucesso!")


Extraindo dados de: characters


Baixando dados: 100%|██████████| 15/15 [00:33<00:00,  2.24s/it]



Extraindo dados de: comics


Baixando dados: 100%|██████████| 643/643 [37:12<00:00,  3.47s/it]



Extraindo dados de: events
Erro 500: {}
Erro ao obter dados iniciais.


Buscando relacionamentos: 100%|██████████| 10/10 [00:32<00:00,  3.25s/it]


Relacionamentos extraídos com sucesso!
